In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays

WebIO._IJuliaInit()

In [48]:
l1 = 1.0
l2 = 1.0
lc1 = l1/2
lc2 = l2/2
m1 = 1.0
m2 = 1.0
g = 9.81
I1 = m1*(l1^2)
I2 = m2*(l2^2)

1.0

In [49]:
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [50]:
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
doublependulum = parse_urdf(urdfpath, floating=false)
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [117]:
using SymPy
sympy.init_printing(use_unicode=True)
@syms x1, y1, x1d, y1d
@syms q1()::real t
@syms q2()::real t
@syms q1d()::real t
@syms q2d()::real t
@syms q1dd()::real t
@syms q2dd()::real t

x1 = l1*sympy.cos(q1(t))
y1 = l1*sympy.sin(q1(t))
x2 = l1*sympy.cos(q1(t)) + l2*sympy.cos(q1(t) + q2(t))
y2 = l1*sympy.sin(q1(t)) + l2*sympy.sin(q1(t) + q2(t))

x1d = diff(x1, t)
y1d = diff(y1, t)
x2d = diff(x2, t)
y2d = diff(y2, t)

U = m1*g*y1 + m2*g*y2
v1 = x1d^2 + y1d^2
v2 = x2d^2 + y2d^2
T = 0.5*m1*v1^2 + 0.5*m2*v2^2

# Le Lagrangian
L = sympy.trigsimp(T - U)
L = L.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
L = L.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d

# Lagrange-Euler Equation
LE1 = diff(diff(L, q1d(t)), t) - diff(L, q1(t))  
LE2 = diff(diff(L, q2d(t)), t) - diff(L, q2(t))
LE = [LE1, LE2]
LE = sympy.trigsimp(LE)

# subs first derivative
LE = LE.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
# subs second derivative
LE = LE.subs(sympy.Derivative(q1d(t), t), q1dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sympy.Derivative(q2d(t), t), q2dd(t))  # substitute d/dt q2d(t) with q2dd
LE = sympy.simplify(LE)
# Generalized mass matrix
# M1 = sympy.Poly(LE[1], q1dd(t)).coeffs()
M11 = collect(expand(LE[1]), q1dd(t)).coeff(q1dd(t))
#=@show M11
M12 = collect(expand(LE[1]), q2dd(t)).coeff(q2dd(t))
M21 = collect(expand(LE[2]), q1dd(t)).coeff(q1dd(t))
M22 = collect(expand(LE[2]), q2dd(t)).coeff(q2dd(t))
M = [[M11, M12] [M21, M22]]

# Coriolis Matrix
C11 = collect(expand(LE[1]), q1dd(t)).coeff(q1dd(t))
C12 = collect(expand(LE[1]), q2dd(t)).coeff(q2dd(t))
C21 = collect(expand(LE[2]), q1dd(t)).coeff(q1dd(t))
C22 = collect(expand(LE[2]), q2dd(t)).coeff(q2dd(t))

@show size(M)
=#

L = sympy.trigsimp(T - U) = 2.0*(cos(q2(t))*Derivative(q1(t), t)^2 + cos(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + Derivative(q1(t), t)^2 + Derivative(q1(t), t)*Derivative(q2(t), t) + 0.5*Derivative(q2(t), t)^2)^2 - 9.81*sin(q1(t) + q2(t)) - 19.62*sin(q1(t)) + 0.5*Derivative(q1(t), t)^4


        2       2                  2                         2                
12.0⋅q1d (t)⋅cos (q₂(t)) + 24.0⋅q1d (t)⋅cos(q₂(t)) + 12.0⋅q1d (t) + 8.0⋅q1d(t)

           2                                                                  
⋅q2d(t)⋅cos (q₂(t)) + 24.0⋅q1d(t)⋅q2d(t)⋅cos(q₂(t)) + 16.0⋅q1d(t)⋅q2d(t) + 6.0

    2                        2   
⋅q2d (t)⋅cos(q₂(t)) + 6.0⋅q2d (t)

In [6]:
function f_1(x)
    # double pendulum dynamics
    
    q1 = x[1]
    q1̇ = x[2]
    q2 = x[3]
    q2̇ = x[4]
    
    M = zeros(2, 2)
    M[1, 1] = I1 + I2 + m2*l1^2 + 2*m2*l1*lc2*cos(q2)
    M[1, 2] = I2 + m2*l1*lc2*cos(q2)
    M[2, 1] = I2 + m2*l1*lc2*cos(q2)
    M[2, 2] = I2
    
    C = zeros(2, 2)
    C[1, 1] = -2*m2*l1*lc2*sin(q2)*q2̇ 
    C[1, 2] = -m2*l1*lc2*sin(q2)*q2̇ 
    C[2, 1] = m2*l1*lc2*sin(q2)*q1̇
    C[2, 2] = 0
    
    τg = zeros(2)
    τg[1] = -m1*g*lc1*sin(q1) - m2*g*(l1*sin(q1) + lc2*sin(q1+q2))
    τg[2] = -m2*g*lc2*sin(q1+q2)
    
    q_d = zeros(2)
    q_d[1] = q1̇ 
    q_d[2] = q2̇ 
    
    q_dd = M\(τg - C*q_d)
    
    q1̈  = q_dd[1]
    q2̈  = q_dd[2]
    
    ẋ = zeros(4)
    ẋ[1] = q1̇  # q1 dot
    ẋ[2] = q1̈  # q1 double dot
    ẋ[3] = q2̇  # q2 dot
    ẋ[4] = q2̈  # q2 double dot
    
end

f_1 (generic function with 1 method)

In [7]:
# stolen from https://github.com/RoboticExplorationLab/RobotZoo.jl/blob/master/src/acrobot.jl

function f_2(x)
    u = [0, 0]
    g = 9.81
    # m1,m2 = model.m
    l1,l2 = l1, l2
    J1,J2 = I1, I2
    θ1,    θ2    = x[1], x[3]
    θ1dot, θ2dot = x[2], x[4]
    s1,c1 = sincos(θ1)
    s2,c2 = sincos(θ2)
    c12 = cos(θ1 + θ2)

    # mass matrix
    m11 = m1*l1^2 + J1 + m2*(l1^2 + l2^2 + 2*l1*l2*c2) + J2
    m12 = m2*(l2^2 + l1*l2*c2 + J2)
    m22 = l2^2*m2 + J2
    M = @SMatrix [m11 m12; m12 m22]

    # bias term
    tmp = l1*l2*m2*s2
    b1 = -(2 * θ1dot * θ2dot + θ2dot^2)*tmp
    b2 = tmp * θ1dot^2
    B = @SVector [b1, b2]

    # friction
    c = 0
    C = @SVector [c*θ1dot, c*θ2dot]

    # gravity term
    g1 = ((m1 + m2)*l2*c1 + m2*l2*c12) * g
    g2 = m2*l2*c12*g
    G = @SVector [g1, g2]

    # equations of motion
    τ = @SVector [0, u[1]]
    θddot = M\(τ - B - G - C)
    return @SVector [θ1dot, θddot[1], θ2dot, θddot[2]]
end


f_2 (generic function with 1 method)

In [8]:
function rk4_step(f,xk,h)
    
    #TODO: implement rk4
    # xn = zeros(length(xk))
    
    a = [0 0 0 0; 
         0.5 0 0 0; 
         0 0.5 0 0; 
         0 0 1 0]
    b = [1/6, 1/3, 1/3, 1/6]
    f1 = f(xk)
    f2 = f(xk .+ h*a[2, 1]*f1)
    f3 = f(xk .+ h*a[3, 1]*f1 .+ h*a[3, 2]*f2)
    f4 = f(xk .+ h*a[4, 1]*f1 .+ h*a[4, 2]*f2 .+ h*a[4, 3]*f3)
    xn = xk .+ h*(b[1]*f1 .+ b[2]*f2 .+ b[3]*f3 .+ b[4]*f4)
    
    return xn
end

rk4_step (generic function with 1 method)

In [9]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [21]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= rk4_step(f_1, xtraj[:,k], h)
    end
end

simulate! (generic function with 1 method)

In [22]:
x0 = [-150*(pi/180); 0.0; 120*(pi/180); 0.0]
xtraj = zeros(4,N)
xtraj[:,1] = x0;

In [23]:
simulate!(xtraj, N)

In [24]:
q = configuration(state)
v = velocity(state)

2-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0

In [25]:
q1 = -xtraj[1, :] .- 150*(pi/180)
q2 = -xtraj[3, :] .+ 120*(pi/180)
qs = convert(AbstractArray{Float64, 2}, [q1 q2]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

10001-element Vector{Vector{Float64}}:
 [0.0, 0.0]
 [-2.5856501677523624e-6, 4.063166349599356e-6]
 [-1.03425951292202e-5, 1.6252678297412615e-5]
 [-2.327081826036803e-5, 3.656857453870899e-5]
 [-4.1370291854470054e-5, 6.501091956812033e-5]
 [-6.464097712566286e-5, 0.00010157980367964115]
 [-9.308282420805725e-5, 0.00014627534297151357]
 [-0.0001266957721615114, 0.00019909767934755962]
 [-0.00016547974897251905, 0.0002600469805225103]
 [-0.00020943467155865036, 0.0003291234400273346]
 [-0.00025856044577343695, 0.00040632727721368056]
 [-0.0003128569664112568, 0.0004916587372618686]
 [-0.0003723241172117753, 0.0005851180911875531]
 ⋮
 [-0.16084793493727467, 0.3443831085948763]
 [-0.15921607133942173, 0.3431778409036843]
 [-0.15758906679196194, 0.3419820298361238]
 [-0.15596692826277847, 0.34079566943094286]
 [-0.1543496627196066, 0.3396187537424742]
 [-0.15273727712981966, 0.3384512768406629]
 [-0.15112977846021902, 0.3372932328110949]
 [-0.1495271736768231, 0.3361446157550252]
 [-0.147

In [26]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(doublependulum, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x31, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))))))), Set{Any}(), ip"127.0.0.1", 8701))

In [27]:
# set_configuration!(mvis, [x0[1]-150*(pi/180), x0[3]+120*(pi/180)])
set_configuration!(mvis, [-x0[1]-150*(pi/180), -x0[3]+120*(pi/180)])
# set_configuration!(mvis, [0, 0])

In [28]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);
animation = Animation(mvis, ts, -q_array)
setanimation!(mvis, animation)